In [50]:
import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df = pd.read_csv("/Users/franckglastre/emaildata_100000_0.csv", nrows=200)
#df = pd.read_csv("emaildata_100000_0.csv", nrows=200)
df.head()

,Unnamed: 0,date,sender,recipient1,subject,text
0,0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


In [51]:
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})


In [52]:
# préparation du corpus de mots avec fonction de nettoyage
import re
def data_cleanstr(string):
    r = re.compile(r'[a-zA-Z0-9.-]*@[a-zA-Z0-9.-]*')
    tok=r.sub('mail', string)
    
    r = re.compile(r"([^a-zA-Z]+)")
    tok1=r.sub(' ', tok)
    return tok1
    
    

In [53]:
# préparation fonction stopwords


def remove_stop_words(text):
    import spacy
    from spacy.lang.en.stop_words import STOP_WORDS
    stopwords=list(STOP_WORDS)
    lst=[]

    for token in text.split():
        if token.lower() not in stopwords:    #checking whether the word is not 
            lst.append(token)                    #present in the stopword list.      
            result = ' '.join(lst)
    return result

In [54]:
# tokenisation du corpus de mots, utilisation de la fonction de nettoyage et mise en minuscules
texte=[]
phrases=""
from nltk.tokenize import word_tokenize

for i in df['body']:
    phrases+=i  
a=data_cleanstr(str(phrases))
a_sans=remove_stop_words(a)
mots=word_tokenize(a)
mots_sans=word_tokenize(a_sans)

# liste contenant le corpus de mots, le premier avec les stopwords, le second sans les stopswords --> épuré des chiffres,
# et des signes de punctuations ou autres

mots=[i.lower() for i in mots]
mots_sans=[i.lower() for i in mots_sans]



In [55]:
# vectorisation du corpus de mots

from sklearn.feature_extraction.text import CountVectorizer

# Créer un vectorisateur
vectorizer = CountVectorizer()

# Appliquer Bag of words à la variable tokens
vectorizer.fit_transform(mots).todense()

# Récupération des tokens
tokenized = vectorizer.vocabulary_
token=dict(sorted(tokenized.items(),key= lambda x:x[1],reverse=True))

print(token)




{'zuniga': 3359, 'zub': 3358, 'zoning': 3357, 'zimin': 3356, 'zavala': 3355, 'zarsky': 3354, 'zarin': 3353, 'yzaguirre': 3352, 'yy': 3351, 'yuan': 3350, 'yowman': 3349, 'yourself': 3348, 'yours': 3347, 'your': 3346, 'you': 3345, 'yolanda': 3344, 'yj': 3343, 'yet': 3342, 'yesterday': 3341, 'yes': 3340, 'yellow': 3339, 'years': 3338, 'year': 3337, 'yates': 3336, 'yahoo': 3335, 'xls': 3334, 'www': 3333, 'wukasch': 3332, 'wsccnav': 3331, 'wscc': 3330, 'wrote': 3329, 'wrong': 3328, 'written': 3327, 'write': 3326, 'would': 3325, 'worth': 3324, 'worried': 3323, 'world': 3322, 'worksheet': 3321, 'works': 3320, 'working': 3319, 'worked': 3318, 'work': 3317, 'word': 3316, 'wood': 3315, 'wong': 3314, 'wolfe': 3313, 'without': 3312, 'within': 3311, 'withdrawing': 3310, 'withdrawals': 3309, 'withdrawal': 3308, 'with': 3307, 'wish': 3306, 'winter': 3305, 'winner': 3304, 'winmail': 3303, 'window': 3302, 'wind': 3301, 'win': 3300, 'willing': 3299, 'williams': 3298, 'william': 3297, 'will': 3296, 'wilc

In [80]:
# essai de vectorisation 

#print(vectorizer.transform(["laissez-vous","dd"]).toarray())
len(df['body'])

200

In [109]:
# vectorisation de chaque email 1 par 1 et enregistrement dans une colonne dédiée body_vect, au passage création de body_clean
def vectorisation(df):
    texte=[]
    df['body_clean']=""
    for i,j in enumerate(df['body']):
        text=data_cleanstr(str(j))
        df.loc[i,"body_clean"]=text
        df.loc[i,"body_vect"]=[vectorizer.transform([text]).toarray()]
        
        
 
# liste contenant le corpus de mots, le premier avec les stopwords, le second sans les stopswords --> épuré des chiffres,
# et des signes de punctuations ou autres

vectorisation(df)

df.head(3)


,Unnamed: 0,date,from,to,header,body,body_vec,body_clean,body_vect
0,0.0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']",,Here is our forecast,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1.0,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak...",,Traveling to have a business meeting takes th...,"[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,2.0,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']",,test successful way to go,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [120]:
df['body_clean'][1]

' Traveling to have a business meeting takes the fun out of the trip Especially if you have to prepare a presentation I would suggest holding the business plan meetings here then take a trip without any formal business meetings I would even try and get some honest opinions on whether a trip is even desired or necessary As far as the business meetings I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not Too often the presenter speaks and the others are quiet just waiting for their turn The meetings might be better if held in a round table discussion format My suggestion for where to go is Austin Play golf and rent a ski boat and jet ski s Flying somewhere takes too much time '